<a href="https://colab.research.google.com/github/AlexeyTri/MLSeminars/blob/main/MLHSE/STEPIK/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Boostings_task_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Прогнозируем задержки самолетов 

In [ ]:
!pip install catboost lightgbm optuna -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 3.4 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import optuna

from catboost import CatBoostClassifier 
from lightgbm import LGBMClassifier

In [ ]:
RANDOM_STATE = 111
DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

In [ ]:
data = pd.read_csv(DATASET_PATH)

X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'] == 'Y'

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


In [ ]:
np.where(X.dtypes != int)

(array([0, 1, 2, 4, 5, 6]),)

Создайте список номеров колонок с категориальными признаками для бустингов

## Quiz 
Какой длины получился список?

(подсказка: колонка `DepTime` числовая)

In [ ]:
cat_features = [0, 1, 2, 4, 5, 6]

Разобъем данные на обучение и контроль

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [ ]:
Xtrain.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
41207,c-4,c-18,c-1,1457,CO,EWR,TPA,998
28283,c-11,c-1,c-2,1225,UA,DEN,BOS,1754
34619,c-6,c-16,c-5,1650,YV,IAD,CAE,401
8789,c-5,c-18,c-4,923,AA,SLC,DFW,988
38315,c-2,c-14,c-2,1839,AA,STL,SAN,1558


## Модели с параметрами по умолчанию

Обучите CatBoost с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [ ]:
# your code here
ctb = CatBoostClassifier()

ctb.fit(Xtrain, ytrain, cat_features=cat_features)

In [ ]:
pred_ctb = ctb.predict(Xtest)

In [ ]:
pred_ctb_proba = ctb.predict_log_proba(Xtest)

In [ ]:
pred_ctb_new = pred_ctb.loc[:, 0].map({'False': False, 'True': True})

In [ ]:
pred_ctb_proba[:, 1]

array([-1.75342269, -2.42810111, -0.95201183, ..., -1.89488467,
       -2.69849154, -2.5462616 ])

In [ ]:
roc_auc_score(ytest, pred_ctb_proba[:, 1])

0.7670858940177856

Обучите LightGBM с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [ ]:
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Xtrain[c] = Xtrain[c].astype('category')
        Xtest[c] = Xtest[c].astype('category')

In [ ]:
# your code her
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier()

lgbm.fit(Xtrain, ytrain)

LGBMClassifier()

In [ ]:
predict_lgbm = lgbm.predict_proba(Xtest)

In [ ]:
predict_lgbm

array([[0.78626587, 0.21373413],
       [0.92100173, 0.07899827],
       [0.36394802, 0.63605198],
       ...,
       [0.89451522, 0.10548478],
       [0.93734167, 0.06265833],
       [0.88555497, 0.11444503]])

In [ ]:
roc_auc_score(ytest, predict_lgbm[:, 1])

0.7341149074685321

## Optuna

Выделим дополнительную валидационную выборку.

In [ ]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=RANDOM_STATE)

Создайте функцию objective_lgbm, в которой среди гиперпараметров

* num_leaves = trial.suggest_int("num_leaves", 10, 100)
* n_estimators = trial.suggest_int("n_estimators", 10, 1000)

подберите оптимальные, обучая LGBM на Xtrain_new, ytrain_new и проверяя качество (ROC-AUC) на Xval.

Используйте 30 эпох обучения Optuna.


In [ ]:
# your code here
def objective_lgbm(trial):
    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)
    max_depth = trial.suggest_int('max_depth', 1, 100)
    cv =trial.suggest_int('cv', 3, 10)

    score = cross_val_score(LGBMClassifier(num_leaves=num_leaves, n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth), Xtrain_new, ytrain_new, cv=cv, scoring='roc_auc', n_jobs=-1).mean()
    
    return score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_lgbm, n_trials=30)
study.best_params

[I 2023-04-03 21:04:11,771] A new study created in memory with name: no-name-a9fd33c2-7786-46c2-9157-27ba980c9014
[I 2023-04-03 21:04:23,550] Trial 0 finished with value: 0.7033277587441731 and parameters: {'num_leaves': 38, 'n_estimators': 496, 'learning_rate': 0.00048430933833997625, 'max_depth': 4, 'cv': 4}. Best is trial 0 with value: 0.7033277587441731.
[I 2023-04-03 21:04:49,693] Trial 1 finished with value: 0.6896272867741174 and parameters: {'num_leaves': 12, 'n_estimators': 703, 'learning_rate': 1.0635926973783392e-05, 'max_depth': 90, 'cv': 7}. Best is trial 0 with value: 0.7033277587441731.
[I 2023-04-03 21:05:12,024] Trial 2 finished with value: 0.696501892228065 and parameters: {'num_leaves': 92, 'n_estimators': 280, 'learning_rate': 3.70838275826695e-05, 'max_depth': 75, 'cv': 8}. Best is trial 0 with value: 0.7033277587441731.
[I 2023-04-03 21:05:57,608] Trial 3 finished with value: 0.7184088460519027 and parameters: {'num_leaves': 49, 'n_estimators': 969, 'learning_rate

{'num_leaves': 73,
 'n_estimators': 331,
 'learning_rate': 0.016578305349341203,
 'max_depth': 49,
 'cv': 10}

Обучите модель с найденными гиперпараметрами на Xtrain, ytrain и оцените ROC-AUC на тестовых данных.

In [ ]:
# your code here
lgbm_new = LGBMClassifier(**{'num_leaves': 73,
 'n_estimators': 331,
 'learning_rate': 0.016578305349341203,
 'max_depth': 49})
lgbm_new.fit(Xtrain, ytrain)

LGBMClassifier(learning_rate=0.016578305349341203, max_depth=49,
               n_estimators=331, num_leaves=73)

In [ ]:
roc_auc_score(ytest, lgbm_new.predict_proba(Xtest)[:, 1])

0.7362538105429639

## Quiz

Чему равно количество листьев в LGBM после подбора гиперпараметров? 

## Работа над улучшением модели

* Попробуйте при помощи Optuna подобрать и другие гиперпарамтеры
* Также подберите гиперпараметры у CatBoost (а не только у LightGBM)

In [ ]:
def objective_lgbm_new(trial):
    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    min_split_gain = trial.suggest_float('min_split_gain', 0.01, 0.07)
    reg_alpha = trial.suggest_float('reg_alpha', 0., 0.5)
    reg_lambda = trial.suggest_float('reg_lambda', 0., 0.5)
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)
    max_depth = trial.suggest_int('max_depth', 1, 100)

    score = cross_val_score(LGBMClassifier(min_split_gain=min_split_gain, reg_alpha=reg_alpha, reg_lambda=reg_lambda,  num_leaves=num_leaves, n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, ), Xtrain_new, ytrain_new, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    
    return score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_lgbm_new, n_trials=50)
study.best_params

[I 2023-04-03 20:46:53,757] A new study created in memory with name: no-name-42207e9b-ab83-47a3-9047-78686d4c04f2
[I 2023-04-03 20:47:10,549] Trial 0 finished with value: 0.6938685160495712 and parameters: {'num_leaves': 46, 'min_split_gain': 0.02342522869080089, 'reg_alpha': 0.19146774720049003, 'reg_lambda': 0.4840314835618635, 'n_estimators': 580, 'learning_rate': 3.363897023081196e-05, 'max_depth': 12}. Best is trial 0 with value: 0.6938685160495712.
[I 2023-04-03 20:47:27,844] Trial 1 finished with value: 0.7058846373221127 and parameters: {'num_leaves': 24, 'min_split_gain': 0.06325037651014814, 'reg_alpha': 0.44453182026861193, 'reg_lambda': 0.3349176118042415, 'n_estimators': 854, 'learning_rate': 0.0003290543652844921, 'max_depth': 59}. Best is trial 1 with value: 0.7058846373221127.
[I 2023-04-03 20:47:33,613] Trial 2 finished with value: 0.7005668525333638 and parameters: {'num_leaves': 36, 'min_split_gain': 0.03632914365276334, 'reg_alpha': 0.23436499391422982, 'reg_lambda'

{'num_leaves': 10,
 'min_split_gain': 0.0535049493971278,
 'reg_alpha': 0.266102172629794,
 'reg_lambda': 0.03211895145286614,
 'n_estimators': 800,
 'learning_rate': 0.00910679016073083,
 'max_depth': 80}

In [ ]:
lgbm_new = LGBMClassifier(**study.best_params)
lgbm_new.fit(Xtrain, ytrain)

LGBMClassifier(learning_rate=0.00910679016073083, max_depth=80,
               min_split_gain=0.0535049493971278, n_estimators=800,
               num_leaves=10, reg_alpha=0.266102172629794,
               reg_lambda=0.03211895145286614)

In [ ]:
roc_auc_score(ytest, lgbm_new.predict_proba(Xtest)[:, 1])

0.735020465135668

In [ ]:
# your code here

## Quiz

Поделитесь своими результатами!